In [1]:
# Boilerplate
import numpy as np
import tmv3 as tm
from tmv3 import Lt_from_hc_GammaChi  # Necessary when loading previously-saved function objects

# Import changes from editing TableMaker
import importlib
importlib.reload(tm)

assert callable(Lt_from_hc_GammaChi), "Lt_from_hc_GammaChi must be callable"

In [2]:
import tmv3_class as tmv3c
importlib.reload(tmv3c)

<module 'tmv3_class' from '/Users/cougar/jaredporter/inferno/codes/TableMakerMain/postGit/tmv3_class.py'>

In [3]:
##### Set parameters for the table
numXim = 150
ximLfrac = 0.2
ximGfrac = 0.5
numXiv = 30

# Set filepath
path = r'./data/ChiGammaTablev3'
file_pattern = r'flm_.*.dat$'

# Adjust data parameters (change depending on how ignis was run)
tvals = np.arange(0,14,1)
Lvals = np.arange(0,26,1)
gammaValues = [0., 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65]
Lbounds = [min(Lvals), max(Lvals)]
tbounds = [min(tvals), max(tvals)]
path_to_hsens = './data/ChiGammaTablev3/hsens.dat'

Test using the object-oriented version first...

In [4]:
# Class for the table
tableDemo = tmv3c.table(path, Lvals, tvals, nxim=5, nxiv=5, ximLfrac=ximLfrac, ximGfrac=ximGfrac,
                        gammaValues=gammaValues, flmt_file_pattern=r'^flm.*.dat$')

In [5]:
# Create a temperature table
tableDemo.phi_mvhc('T')

Parsing flamelet data
Parsing data files...
Completed data import (parse_data)

Beginning parallel table creation...
phi_funcs for h created using 364 files.
phi_funcs for c created using 364 files.
phi_funcs for T created using 364 files.
Creating phi_funcs for h...
Creating phi_funcs for c...
h table 0% complete.
Creating phi_funcs for T...
c table 0% complete.
T table 0% complete.
h table 20% complete.
c table 20% complete.
T table 20% complete.


/Users/cougar/jaredporter/inferno/codes/TableMakerMain/postGit/LiuInt.py:133: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  p2 = quad(ϕP, ϵ, 1-ϵ, args = (a, b), points = [ξm,])[0]   # ϵ   < ξ < 1-ϵ


c table 40% complete.
h table 40% complete.
T table 40% complete.
c table 60% complete.
c table 80% complete.
Lookup table for phi = c completed.
h table 60% complete.
T table 60% complete.
h table 80% complete.
Lookup table for phi = h completed.
T table 80% complete.
Lookup table for phi = T completed.


({'h': <function tmv3_class.table.phi_mvhc.<locals>.create_phi_table.<locals>.phi_table(xim, xiv, h, c, useStoredSolution=True, solver='gammachi', detailedWarn=False, maxIter=100, saveSolverStates=False, LstepParams=[0.25, 0.01, 0.003], tstepParams=[0.25, 9.5, 0.02])>,
  'c': <function tmv3_class.table.phi_mvhc.<locals>.create_phi_table.<locals>.phi_table(xim, xiv, h, c, useStoredSolution=True, solver='gammachi', detailedWarn=False, maxIter=100, saveSolverStates=False, LstepParams=[0.25, 0.01, 0.003], tstepParams=[0.25, 9.5, 0.02])>,
  'T': <function tmv3_class.table.phi_mvhc.<locals>.create_phi_table.<locals>.phi_table(xim, xiv, h, c, useStoredSolution=True, solver='gammachi', detailedWarn=False, maxIter=100, saveSolverStates=False, LstepParams=[0.25, 0.01, 0.003], tstepParams=[0.25, 9.5, 0.02])>},
 {})

...then using the older package-based version.

In [6]:
result_demo = tm.phi_mvhc(path, Lvals, tvals, file_pattern = file_pattern, phi = ['T'], 
                        numXim = 5, numXiv = 5, parallel = True,
                        ximLfrac = ximLfrac, ximGfrac = ximGfrac)

funcs_demo, tableInfo_demo = result_demo
Tfunc_demo = funcs_demo[0]
h_table_demo = tableInfo_demo[0][0]
h_indices_demo = tableInfo_demo[0][1]
c_table_demo = tableInfo_demo[1][0]
c_indices_demo = tableInfo_demo[1][1]
T_table_demo, T_indices_demo = tableInfo_demo[2][0], tableInfo_demo[2][1]

Completed data import ('get_data_files')
Beginning parallel table creation...
phi_funcs for h created using 364 files.
h table 0% complete.
phi_funcs for c created using 364 files.
c table 0% complete.
phi_funcs for T created using 364 files.
T table 0% complete.
h table 20% complete.
c table 20% complete.
T table 20% complete.


/Users/cougar/jaredporter/inferno/codes/TableMakerMain/postGit/LiuInt.py:133: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  p2 = quad(ϕP, ϵ, 1-ϵ, args = (a, b), points = [ξm,])[0]   # ϵ   < ξ < 1-ϵ


c table 40% complete.
h table 40% complete.
T table 40% complete.
c table 60% complete.
c table 80% complete.
Lookup table for phi = c completed.
h table 60% complete.
T table 60% complete.
h table 80% complete.
Lookup table for phi = h completed.
T table 80% complete.
Lookup table for phi = T completed.


Now, figure out where the maximum discrepancy between the two is:

In [7]:
from scipy.optimize import differential_evolution
import numpy as np

h = 0.0
c = 0.0

def neg_abs_diff(mv):
    xim, xivRel = mv
    xim = np.clip(xim, 0.0, 1.0)
    xivRel = np.clip(xivRel, 0.0, 1.0)
    xiv = xim * (1 - xim) * xivRel

    T1 = tableDemo.phi_mvhc_funcs['T'](xim, xiv, h, c)
    T2 = Tfunc_demo(xim, xiv, h, c, gammaValues=gammaValues, path_to_hsens=path_to_hsens)

    return -abs(T1 - T2)

bounds = [(0.0, 1.0), (0.0, 1.0)]

result = differential_evolution(neg_abs_diff, bounds, seed=0, 
                                maxiter=10, tol=1e-2)

print(f"Max absolute difference: {-result.fun} K")
print("Location of max difference:", result.x)

/Users/cougar/jaredporter/inferno/codes/TableMakerMain/postGit/LiuInt.py:133: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  p2 = quad(ϕP, ϵ, 1-ϵ, args = (a, b), points = [ξm,])[0]   # ϵ   < ξ < 1-ϵ


Max absolute difference: 0.0 K
Location of max difference: [0.5652136 0.1842965]


Validate Differential Evolution's solution

In [8]:
xim, xivRel = result.x
xiv = xim * (1 - xim) * xivRel
T1 = tableDemo.phi_mvhc_funcs['T'](xim, xiv, h, c)
T2 = Tfunc_demo(xim, xiv, h, c, gammaValues=gammaValues, path_to_hsens=path_to_hsens)
print(f"T1 = {T1}, T2 = {T2}, Difference = {abs(T1 - T2)}")

T1 = [235.17307522], T2 = [235.17307522], Difference = [0.]


Alternative iterative method for finding discrepancies

In [9]:
ximArr = np.arange(0, 1, 97)
xivArr = np.linspace(0, 1, 98)
h = 0.0
c = 0.0
for i in range(len(ximArr)):
    for j in range(len(xivArr)):
        xim = ximArr[i]
        xiv = xivArr[j]*xim*(1 - xim)

        classVal = tableDemo.phi_mvhc_funcs['T'](xim, xiv, h, c)
        pakijVal = Tfunc_demo(xim, xiv, h, c, gammaValues=gammaValues, path_to_hsens=path_to_hsens)
        
        if not np.isclose(classVal, pakijVal):
            print(f"Discrepancy found at xim={xim}, xiv={xiv}: classVal-pakijVal={classVal-pakijVal}")
            print()